# Homework 1 Part 4

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri
#### Notebooks Prepared By: Zeinab Sadat Taghavi, Hamed Jamshidian, Seyed Mohammad Reza Modarres

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between `## Your code begins ##` and `## Your code ends ##`) with the appropriate details.


## Introduction

In this notebook you have to use [Adapter Hub](https://docs.adapterhub.ml/overview.html) to define and train your adapters.

### Requirements

In [2]:
%%capture
! pip install datasets adapter-transformers

In [3]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00


### Imports

In [4]:
from tqdm.notebook import tqdm
from IPython import display

import numpy as np
import pandas as pd
import math

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import T5TokenizerFast, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

### Constants

We will use `t5-small` as our base model from Hugging Face ([HF_Link](https://huggingface.co/t5-small)). And we use `32` as our adapter bottleneck size.

In [5]:
#####################################
###### DO NOT CHANGE THIS CELL ######
#####################################

BASE_MODEL_NAME = 't5-small'

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataset

### Load dataset

`imdb` dataset is a famouns NLP sentiment dataset. Each row of data is either `negative` or `positive`.

In [6]:
dataset = load_dataset('imdb')
dataset.pop('unsupervised')
print(dataset)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


### Define related functions

Because `T5` model is a sequence to sequence model we should map our labels to label_names before training and doing vice versa duing calculating metrics.

The functions `id2label` and `label2id` are defined to do this.

In [7]:
def id2label(ids):
    label_names = ['negative', 'positive']
    return [label_names[id] for id in ids]

def label2id(labels):
    label_names_dict = {
        'negative': 0,
        'positive': 1
    }
    return [
        label_names_dict.get(label, 2)
        for label in labels
    ]

# Tokenizer

### Load tokenizer

In [8]:
tokenizer = T5TokenizerFast.from_pretrained(BASE_MODEL_NAME)

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

(…)ce.co/t5-small/resolve/main/spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…).co/t5-small/resolve/main/tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

### Process dataset using tokenizer

In this step we will getting our dataset ready for training.

We preprocess tokenize our `text` and `label`.

In [9]:
def preprocess_input(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    return text

def map_function(row):
    processed_input = [
        preprocess_input(text)
        for text in row['text']
    ]
    input_info = tokenizer(processed_input, truncation=True, max_length=256)
    output_info = tokenizer(id2label(row['label']))
    return {
        **input_info,
        'labels': output_info.input_ids
    }


dataset = dataset.map(map_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# Model

### Load model and create adapter

In next part create `PfeifferConfig` by considering `BOTTLENECK_SIZE`.

Complete the next cell using methods of `train_adapter` and `add_adapter`.

Report final test data performance.s

Read [this page](https://docs.adapterhub.ml/training.html) for more information.

In [10]:
from transformers import PfeifferConfig
from transformers import TrainingArguments, AdapterTrainer
from sklearn.metrics import accuracy_score
import torch
from transformers.adapters import T5AdapterModel, AutoAdapterModel

def recreate_model_with_bottleneck(bottleneck):

  model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)
  # model = AutoAdapterModel.from_pretrained(base_model)


  model.freeze_model(freeze=True)

  adapter_name = f"llm_adapter_peft_btlnck_{str(bottleneck)}"
  config = PfeifferConfig(reduction_factor = model.config.d_model / bottleneck, output_adapter = True)

  model.add_adapter(adapter_name, config=config, set_active=True)

  model = model.to(DEVICE)

  return model, adapter_name


# Train and evaluate

Complete next part to train your peft using `AdapterTrainer`.

### Fine-Tuning With Bottleneck Size = 8

In [11]:
BOTTLENECK_SIZE = 8

model, adapter_name = recreate_model_with_bottleneck(BOTTLENECK_SIZE)

(…)ace.co/t5-small/resolve/main/config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

(…)mall/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

train_loader = torch.utils.data.DataLoader(
    dataset['train'],
    batch_size=BATCH_SIZE,
    collate_fn=col_fn,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset['test'],
    batch_size=BATCH_SIZE,
    collate_fn=col_fn
)

In [13]:
def train_loop(model, loader, optimizer):
    model.train()

    batch_losses = []

    for row in tqdm(loader, desc='Training:'):
        optimizer.zero_grad()

        out = model.base_model(**row.to(model.device))
        loss = out.loss

        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)

    loss_value = np.mean(batch_losses)
    return {'train_loss': loss_value}

def _predict(model, row):
    return model.generate(
        input_ids=row.input_ids,
        attention_mask=row.attention_mask,
        max_length=5
    )

def tokenizer_ids_to_label(all_input_ids):
    return tokenizer.batch_decode(all_input_ids, skip_special_tokens=True)

def valid_loop(model, loader, compute_metrics):
    model.eval()

    all_true = []
    all_pred = []

    with torch.no_grad():
        for row in tqdm(loader, desc='Validating:'):
            row.to(model.device)
            pred = _predict(model, row)

            all_true += row.labels.detach().cpu().tolist()
            all_pred += pred.detach().cpu().tolist()

    all_true = label2id(tokenizer_ids_to_label(all_true))
    all_pred = label2id(tokenizer_ids_to_label(all_pred))

    return {'valid_acc': compute_metrics(y_true=all_true, y_pred=all_pred)}

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
compute_metrics = accuracy_score

In [15]:
def train_model(_model):
  _model = _model.to(DEVICE)
  all_results = []
  for epoch in range(EPOCHS):
      epoch_results = {'epoch': epoch}

      epoch_results.update(
          train_loop(
              model=_model,
              loader=train_loader,
              optimizer=optimizer,
          )
      )

      epoch_results.update(
          valid_loop(
              model=_model,
              loader=test_loader,
              compute_metrics=compute_metrics,
          )
      )
      all_results.append(epoch_results)

      display.clear_output()
      display.display(pd.DataFrame(all_results).set_index('epoch'))
  return all_results

In [16]:
train_results_b8 = train_model(model)

,train_loss,valid_acc
epoch,,
0,0.786092,0.86408
1,0.165856,0.88028
2,0.153489,0.88860
3,0.145989,0.89268
4,0.140664,0.89560
5,0.137107,0.89656
6,0.132820,0.89932
7,0.129383,0.90312
8,0.128629,0.90204


In [17]:
best_score = pd.DataFrame(train_results_b8)['valid_acc'].max() * 100
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")
print('Best model preformance is: %%%.1f' % best_score)

Number of parameters: 60611168
Best model preformance is: %90.4


# Bottleneck effect

Check the model performance with `BOTTLENECK_SIZE = 1` and report the number.

In [18]:
BOTTLENECK_SIZE = 1

model_b1, adapter_name = recreate_model_with_bottleneck(BOTTLENECK_SIZE)

In [19]:
optimizer = torch.optim.AdamW(model_b1.parameters(), lr=LEARNING_RATE)
compute_metrics = accuracy_score

In [20]:
train_results_b1 = train_model(model_b1)

,train_loss,valid_acc
epoch,,
0,1.596445,0.82440
1,0.212451,0.85276
2,0.181254,0.86396
3,0.171166,0.86872
4,0.163382,0.87812
5,0.157763,0.88368
6,0.155499,0.88648
7,0.149246,0.88936
8,0.147901,0.89068


In [21]:
best_score = pd.DataFrame(train_results_b1)['valid_acc'].max() * 100
total_params = sum(p.numel() for p in model_b1.parameters())
print(f"Number of parameters: {total_params}")
print('Best model preformance is: %%%.1f' % best_score)

Number of parameters: 60525068
Best model preformance is: %89.3
